In [9]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By

In [42]:
url = "https://booksy.com/pl-pl/" 

In [43]:
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [47]:
driver = webdriver.Chrome()

In [48]:
driver.get(url)

In [49]:
title = driver.title
title

'Booksy - Fryzjerzy, Barberzy, Kosmetyczki... zarezerwuj wizytę online!'

In [50]:
# cookies accept
driver.find_element(By.ID, 'CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll').click()

In [52]:
# promo close
driver.find_element(By.CSS_SELECTOR, 'button[data-testid="bgc-promo-modal-close-btn"]').click()

In [4]:
placeholder = "Szukaj usług lub biznesów"
input_field = soup.find("input", {"placeholder": placeholder})
print(input_field)

<input class="b-form-control purify_qSSZCtZkJjEp6SxrS55bcQ==" placeholder="Szukaj usług lub biznesów" type="text" value=""/>


In [53]:
input_field = driver.find_element(By.CSS_SELECTOR, "div[data-testid='search-what-desktop'] input")
input_field

<selenium.webdriver.remote.webelement.WebElement (session="6dafc1655fbbc2dbe686eaad5336ba8b", element="f.1AF8C104CF40737EB333C54C9F2F65F3.d.2CB2DD9C37032AC6E1EF7F637E4F5E83.e.3")>

In [60]:
def get_webelement_details(webelement):
    return webelement.tag_name, webelement.get_attribute("class"), webelement.get_attribute("id"), webelement.get_attribute("name"), webelement.get_attribute("placeholder")

In [61]:
get_webelement_details(input_field)

('input',
 'b-form-control purify_qSSZCtZkJjEp6SxrS55bcQ==',
 '',
 '',
 'Szukaj usług lub biznesów')